# Lab: Transfer Learning - Getting Started

We will explore transfer learning concepts

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elephantscale/cool-ML-demos/blob/main/transfer-learning/transfer1-getting-started.ipynb)

### Runtime
~ 15 minutes

### Note
Here we are dealing with real world images.  Processing them will required a lot of compute power.  
If you have access to, switch to **GPU** as run time!

**Instructor** walk through this lab step-by-step and explain

### References
- https://www.tensorflow.org/tutorials/images/transfer_learning
- [lmorony-dlaicourse](https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%206%20-%20Lesson%203%20-%20Notebook.ipynb)


In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
print ('tensorflow version :', tf.__version__)
tf.config.experimental.list_physical_devices()

## TF-GPU Config
The following cell sets TF properties to run on GPU

In [ ]:
## This block is to tweak TF running on GPU
## You may comment this out, if you are not using GPU

## ---- start Memory setting ----
## Ask TF not to allocate all GPU memory at once.. allocate as needed
## Without this the execution will fail with "failed to initialize algorithm" error

from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
## ---- end Memory setting ----

### Custom Model Summary Function
This function only prints out the essentials

In [ ]:
def compact_summary_helper(line):
    matches = ['Model:', 'Total params:', 'Trainable params:', 'Non-trainable params:']
    if any(x in line for x in matches):
        print("*", line)

def print_model_summary_compact(model):
    model.summary(print_fn=compact_summary_helper)
    # print ("* model name:", model.name)
    print ("* # layers: ", len(model.layers))
    


## Step 1 - Download MobileNetv2 Model
Here we will examine MobileNetv2 model.  MobileNetv2 was developed at Google.  It is pre-trained on the ImageNet dataset consisting of 1.4 Million images and 1000 classes.

- paper: [MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/abs/1704.04861)
- [github](https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/README.md)
- [Tensorflow model page](https://www.tensorflow.org/api_docs/python/tf/keras/applications/MobileNetV2)
- [Tensorflow transfer learning guide](https://www.tensorflow.org/tutorials/images/transfer_learning)

When downloading this model, the image dimension should be (224,224,3) to match Imagenet dimensions.

We are also initializing the model with the weights of 'imagenet'

#### TODO
- Notice how many layers we have :-) 
- Look at the final `prediction` layer.  Why does it have 1000 neurons?
- Look at `total parameters` and `total trainable parameters`

In [ ]:
mobilenetv2_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), 
                                               include_top = True,
                                               weights = 'imagenet')

print ("# layers: ", len(mobilenetv2_model.layers))

## Warning: large output!
mobilenetv2_model.summary()

## Step 2 - Try Compact Summary
Let's try our print function

In [ ]:
print_model_summary_compact(mobilenetv2_model)

In [ ]:
## Let's find the size of downloaded model
! du -skh ~/.keras/models/*

## Step 3 -  Freeze Layers
We are going to freeze the model.  
See how many traininable parameters we  have now.  Compare with previous number.

In [ ]:
mobilenetv2_model.trainable = False
print_model_summary_compact(mobilenetv2_model)

## Step 4 - Load Just the Base Model
The last layer of MobileNet is for predicting 1000 classes of ImageNet.  
For our purpose we don't need that.  
So we are going to just load the baes model.  This is achieve by specifying `include_top=False`  
Since we are not using the last layer, we can specify a custom image size other than (224,224,3)

### TODO 
- Compare number of layers of full model and base model
- Also compare number of trainable params of both models

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(160,160,3),
                                               include_top=False,
                                               weights='imagenet')
print ("# layers: ", len(base_model.layers))
base_model.summary()

In [ ]:
print_model_summary_compact(base_model)

## Step 5 - InceptionV3 Model

Inception achieved 79% accuracy in ImageNet data

- [Reference paper](http://arxiv.org/abs/1512.00567)
- [Tensorflow implementation](https://www.tensorflow.org/api_docs/python/tf/keras/applications/InceptionV3)


### TODO
- How many model layers
- How many model parameters
- And inspect the size of the model

In [ ]:
inceptionv3_model = tf.keras.applications.InceptionV3 (input_shape=(299,299,3), 
                                                        include_top = True,
                                                        weights = 'imagenet')

print_model_summary_compact(inceptionv3_model)


In [ ]:
## Model sizes
! du -skh ~/.keras/models/*

## Step 6 - Investigate other Models

You can see other pre-trained models here.  
https://www.tensorflow.org/api_docs/python/tf/keras/applications